In [3]:
from nltk.parse import ChartParser
from nltk.grammar import CFG
import nltk
import csv
import re
import pandas as pd
import argparse
from sklearn.metrics import classification_report
import numpy as np

In [4]:
# parser = argparse.ArgumentParser(description='input file name')
# parser.add_argument('--input',required=True, help="Please add the input file name path and name")
# args = parser.parse_args()

# Read Data from input file given by args.
# df=pd.read_csv(args.input, sep='\t',index_col='id')

# Task A

In [92]:
df=pd.read_csv('../ALL data/Training  Data/subtaskA_data_all.csv', sep=',',index_col='id')
df_label=pd.read_csv('../ALL data/Training  Data/subtaskA_answers_all.csv', sep=',',header=None,index_col=0)
df_label.columns=['label-false']
df=df.join(df_label)


In [93]:
df

,sent0,sent1,label-false
id,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1


In [100]:
df1=df.head()

In [104]:
for index, row in df1.iterrows():  
# tokenize the rows of dataframe (pos column)
    sentence_1 = row['sent0']
    sentence_2 = row['sent1']

    prob_1 = predict(sentence_1, bert_model=model, bert_tokenizer=tokenizer)
    prob_2 = predict(sentence_2, bert_model=model, bert_tokenizer=tokenizer) 
    df1.loc[index,'prob_1']=prob_1
    df1.loc[index,'prob_2']=prob_2
    df1.loc[index,'predicted_false']=np.argmin([prob_1, prob_2]).astype('int64')

    print(np.min([prob_1, prob_2]))
    print('-------------')



0.39989597379060515
-------------
0.42548061000793774
-------------
0.5156204540425442
-------------
0.4335789488698799
-------------
0.5957062266924752
-------------


In [105]:
df1

,sent0,sent1,label-false,prob_1,prob_2,predicted_false
id,,,,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0,0.399896,0.768483,0.0
1,He drinks apple.,He drinks milk.,0,0.610041,0.425481,1.0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1,0.603129,0.515620,1.0
3,A mosquito stings me,I sting a mosquito,1,0.458191,0.433579,1.0
4,A niece is a person.,A giraffe is a person.,1,0.715048,0.595706,1.0


In [112]:
# print accuracy report
print(f"Accuracy is equal to: {((df1['label-false']==df1['predicted_false']).sum()/df1.shape[0])*100} %")

Accuracy is equal to: 80.0 %


# Task B

In [115]:
df=pd.read_csv('../ALL data/Training  Data/subtaskB_data_all.csv', sep=',',index_col='id')
df_label=pd.read_csv('../ALL data/Training  Data/subtaskB_answers_all.csv', sep=',',header=None,index_col=0)
df_label.columns=['label']
df=df.join(df_label)

In [116]:
df

,FalseSent,OptionA,OptionB,OptionC,label
id,,,,,
0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B
1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B
2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A
3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C
4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B
...,...,...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark is bad at making cherry pie,a cherry pie should be big,a cherry pie should be sweet,C
9996,Gloria wears a cat on her head,a hat cannot be worn on a cat's head,a cat cannot be worn on a person's head,the cat is too heavy to be worn on her head,B
9997,Harry went to the barbershop to have his glass...,a barbershop usually don't provide the service...,a barbershop usually repairs computers instead...,the barbershop lacked the necessary tools to r...,A


In [27]:
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [28]:
# from transformers import GPT2LMHeadModel

In [29]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.eval()
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# def score(sentence):
#     tokenize_input = tokenizer.tokenize(sentence)
#     tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
#     loss=model(tensor_input, labels=tensor_input)
#     return -loss[0] * len(tokenize_input)

# a=['there is a book on the desk',
#                 'there is a plane on the desk',
#                         'there is a book in the desk']
# print([score(i) for i in a])

In [30]:
# from transformers import GPT2LMHeadModel, GPT2TokenizerFast
# model_id = 'gpt2'
# # model_id = 'gpt2-large'
# model = GPT2LMHeadModel.from_pretrained(model_id)
# tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

## New section


In [19]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
import math

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [20]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

In [25]:
def predict(text, bert_model, bert_tokenizer):
    # Tokenized input
    # text = "[CLS] I got restricted because Tom reported my reply [SEP]"
    text = "[CLS] " + text + " [SEP]"
    tokenized_text = bert_tokenizer.tokenize(text)
    # text = "[CLS] Stir the mixture until it is done [SEP]"
        #masked_index = 4
    sentence_prob = 1
    for masked_index in range(1,len(tokenized_text)-1):
        # Mask a token that we will try to predict back with `BertForMaskedLM`
        masked_word = tokenized_text[masked_index]
        #tokenized_text[masked_index] = '[MASK]'
        # assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']
        # print (tokenized_text)

        # Convert token to vocabulary indices
        indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        # segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
        length = len(tokenized_text)
        segments_ids = [0 for _ in range(length)]
        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # If you have a GPU, put everything on cuda
        # tokens_tensor = tokens_tensor.to('cuda')
        # segments_tensors = segments_tensors.to('cuda')

        # Load pre-trained model (weights)
        # bert_model = BertForMaskedLM.from_pretrained('bert-large-uncased')
        # bert_model.eval()

        # If you have a GPU, put everything on cuda
        # tokens_tensor = tokens_tensor.to('cuda')
        # segments_tensors = segments_tensors.to('cuda')
        # bert_model.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            predictions = bert_model(tokens_tensor, segments_tensors)

        predictions = torch.nn.functional.softmax(predictions, -1)

        index = bert_tokenizer.convert_tokens_to_ids([masked_word])[0]

        curr_prob = predictions[0, masked_index][index]
        # predict_list = predictions[0, masked_index]
        sentence_prob *= curr_prob
        #tokenized_text[masked_index] = masked_word
    return math.pow(sentence_prob, 1/(len(tokenized_text)-3))
    #return sentence_prob
'''
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('./tmp/swag_output/')
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('./tmp/swag_output/')
model.eval()
'''


"\n# Load pre-trained model tokenizer (vocabulary)\ntokenizer = BertTokenizer.from_pretrained('./tmp/swag_output/')\n# Load pre-trained model (weights)\nmodel = BertForMaskedLM.from_pretrained('./tmp/swag_output/')\nmodel.eval()\n"

In [23]:
# Load pre-trained model tokenizer (vocabulary)
# 'bert-base-uncased'
# 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
# prob = predict(sentence_1, bert_model=model, bert_tokenizer=tokenizer)

0<00:31, 1544198.58B/s]
100%|██████████| 407873900/407873900 [10:02<00:00, 677086.44B/s] 
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpxkypv7uu to cache at /home/yashar/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /home/yashar/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpxkypv7uu
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/yashar/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modelin

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
      

In [26]:
with open("ConcatenatedSentences.txt", "r") as f:
    file = f.readlines()

num = len(file)
count = 0
curr = 0
for i in file:
    label, sentence_1, sentence_2, sentence_3 = i.split("\001")
    print (label[0])
    prob_1 = predict(sentence_1, bert_model=model, bert_tokenizer=tokenizer)
    prob_2 = predict(sentence_2, bert_model=model, bert_tokenizer=tokenizer)
    prob_3 = predict(sentence_3, bert_model=model, bert_tokenizer=tokenizer)
    answer = max(prob_1, prob_2, prob_3)
    if (prob_1 == answer and label[0] == "A") or (prob_2 == answer and label[0] == "B") \
            or (prob_3 == answer and label[0] == "C"):
        count += 1
        print (count)

    print(count, curr)
    curr += 1
print (count/num)

B
1
1 0
A
1 1
B
1 2
B
1 3
A
1 4
B
1 5
C
2
2 6
B
3
3 7
A
3 8
A
3 9
C
4
4 10
A
4 11
B
4 12
C
4 13
B
4 14
A
5
5 15
A
5 16
B
6
6 17
C
7
7 18
B
7 19
A
7 20
B
7 21
B
8
8 22
B
8 23
A
9
9 24
C
9 25
A
10
10 26
A
10 27
C
10 28
C
10 29
B
10 30
C
10 31
B
10 32
A
11
11 33
C
12
12 34
A
12 35
B
12 36
C
13
13 37
B
13 38
B
14
14 39
B
14 40
B
15
15 41
B
15 42
A
15 43
A
15 44
A
15 45
B
16
16 46
C
16 47
C
16 48
A
17
17 49
B
17 50
A
17 51
A
18
18 52
C
18 53
C
18 54
B
18 55
A
19
19 56
B
19 57
C
19 58
B
20
20 59
B
20 60
A
20 61
A
21
21 62
A
21 63
A
21 64
A
21 65
C
21 66
C
21 67
C
22
22 68
B
22 69
B


KeyboardInterrupt: 